In [6]:
using System;
using System.Collections.Concurrent;
using System.Threading;

public interface ICommand
{
    void Execute();
}

public class ServerThread
{
    private ConcurrentQueue<ICommand> commandQueue = new ConcurrentQueue<ICommand>();
    private Thread thread;
    private bool isRunning = true;
    private bool isSoftStopping = false;
    private bool isHardStopping = false;

    public ServerThread()
    {
        thread = new Thread(Run);
        thread.Start();
    }

    public void AddCommand(ICommand command)
    {
        commandQueue.Enqueue(command);
    }

    private void Run()
{
    while (isRunning)
    {
        if (commandQueue.TryDequeue(out ICommand command))
        {
            if (!isSoftStopping && !isHardStopping)
            {
                try
                {
                    command.Execute();
                }
                catch (Exception ex)
                {
                    // Обработка исключения
                    Console.WriteLine($"Исключение в команде: {ex.Message}");
                }
            }
        }
        else if (isSoftStopping)
        {
            isRunning = false;
        }
        else
        {
            Thread.Sleep(1);
        }
    }
}

    public void SoftStop()
    {
        if (Thread.CurrentThread != thread)
        {
            throw new InvalidOperationException("Команда SoftStop может выполняться только в том же потоке.");
        }

        isSoftStopping = true;
    }

    public void HardStop()
    {
        if (Thread.CurrentThread != thread)
        {
            throw new InvalidOperationException("Команда HardStop может выполняться только в том же потоке.");
        }

        isRunning = false;
        thread.Abort();
    }
}

public class HardStopCommand : ICommand
{
    private ServerThread serverThread;

    public HardStopCommand(ServerThread serverThread)
    {
        this.serverThread = serverThread;
    }

    public void Execute()
    {
        serverThread.HardStop();
    }
}

public class SoftStopCommand : ICommand
{
    private ServerThread serverThread;

    public SoftStopCommand(ServerThread serverThread)
    {
        this.serverThread = serverThread;
    }

    public void Execute()
    {
        serverThread.SoftStop();
    }
}

public class CommandWrapper : ICommand
{
    private Action action;

    public CommandWrapper(Action action)
    {
        this.action = action;
    }

    public void Execute()
    {
        action();
    }
}


In [7]:
var serverThread = new ServerThread();
        serverThread.AddCommand(new CommandWrapper(() => Console.WriteLine("Команда номер 1")));
        serverThread.AddCommand(new CommandWrapper(() => Console.WriteLine("Команда номер 2")));
        serverThread.AddCommand(new SoftStopCommand(serverThread));
        serverThread.AddCommand(new CommandWrapper(() => Console.WriteLine("Команда номер 3"))); // Эта команда не доджна быть выполнена
        Thread.Sleep(1000); // Даем потоку авремя на обработку команд
        Console.WriteLine("Основной поток завершается.");

Команда номер 1
Команда номер 2
Основной поток завершается.
